<a href="https://colab.research.google.com/github/That-is/Capstone/blob/master/XGBoost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb

#scikit learn
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [0]:
data = pd.read_csv('/content/drive/My Drive/capstone dataset/Upper_East_Side_South.csv')
#drop unnecessary columns
data.drop(columns = ['Unnamed: 0','location','locationID'], inplace = True)
data.drop(columns=['tpep_pickup_datetime'], inplace=True)
# predictive labels
Y_label = data['15minutes_demand']
# training dataset
X_train = data.drop(columns = ['15minutes_demand'])
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_label,test_size=0.1,random_state=3)
model = xgb.XGBClassifier()
model.fit(x_train,y_train)
value = model.predict(x_test)
value_y_test = y_test.as_matrix()
result = list()
# accuracy criteria
def accuracy(predict_value, true_value):
  for i, j in zip(predict_value, true_value):
    a = abs(i-j)
    result.append(a)
  return result
result_final = accuracy(value,value_y_test)
relative_accuracy = np.sum(result_final)/np.sum(value_y_test)
print(relative_accuracy)

In [0]:
model1 = xgb.XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=300, objective='multi_softmax',
                           gamma = 0, seed = 1, booster='gblinear', silent = True)

In [177]:
model1.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gblinear', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1,
              silent=True, subsample=1, verbosity=1)

In [0]:
value1 = model1.predict(x_test)

In [19]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/capstone dataset/Upper_East_Side_South.csv')
data.iloc[29034:29039,:]

,Unnamed: 0,tpep_pickup_datetime,month,day,hour,minute,demand,15minutes_demand,location,locationID,weekday
29034,29034,2018-06-21 03:54:00,6,21,3,54,0.0,0.0,Upper East Side South,237,3
29035,29035,2018-06-21 03:55:00,6,21,3,55,0.0,0.0,Upper East Side South,237,3
29036,29036,2018-06-21 03:56:00,6,21,3,56,0.0,0.0,Upper East Side South,237,3
29037,29037,2018-06-21 03:57:00,6,21,3,57,0.0,1.0,Upper East Side South,237,3
29038,29038,2018-06-21 03:58:00,6,21,3,58,0.0,2.0,Upper East Side South,237,3


In [90]:
skf = StratifiedKFold(n_splits=5, shuffle= True, random_state = 3)
MAX_ROUNDS = 2000
xgb_gbm_iter1 = []
for i, (train_index, test_index) in enumerate(skf.split(X_train,Y_label)):
  y_train, y_test = Y_label[train_index], Y_label[test_index]
  x_train, x_test = X_train.iloc[train_index,:], X_train.iloc[test_index,:]
  mx_train = xgb.DMatrix(x_train)
  my_test = xgb.DMatrix(y_test)
  mx_test = xgb.DMatrix(x_test)
  my_train = xgb.DMatrix(y_train)
  print('\nFold', i)
#XGboost algorithm
  xgb_gbm = xgb.XGBClassifier(max_depth=6, 
                                n_estimators=MAX_ROUNDS,
                                eval_set=[(x_train, y_train), (x_test, y_test)],
                                learning_rate=0.06,
                                booster = 'gblinear'
                          )
                      
  xgb_gbm.fit(x_train,y_train)
  print( " Best iteration xgboost_gbm = ", xgb_gbm.get_booster().best_iteration)
  pred  = xgb_gbm.predict(x_test)
  ap = average_precision_score(y_test, pred, average='macro', pos_label=1, sample_weight=None)
  print('xgboost ', ap)
  xgb_gbm_ap1 = np.append(xgb_gbm_ap1, ap)
  
  


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)



Fold 0


KeyboardInterrupt: ignored